In [1]:
import torch

# 自动微分模块

PyTorch中具有一个内置的微分引擎`torch.autograd`以支持图的梯度自动计算

考虑最简单的单层神经网络，具有输入x、参数w、偏置b以及损失函数（w、b为参数，CE：损失函数）
| `x`  | -> | *  | -> | +  | -> | `z`  | -> | CE | -> | `loss` |
| :- | :- | :- | :- | :- | :- | :- | :- | :- | :- | :--- |
|    |    | `w`  |    | `b`  |    |    |    | `y`  |    |      |

Tensor类中定义了梯度相关的属性和方法，首先，值记录在属性`data`中，通过布尔类型属性`requires_grad`来判断该张量是否需要梯度计算，计算出的梯度存放在`grad`属性中，计算梯度时需要知道计算图中发生了哪些操作，计算的函数记录在属性`grad_fn`中

如果一个张量是通过另外张量计算得到的，那么若计算部分中的任一张量的`requires_grad`属性是开启的，那么此张量的`requires_grad`属性也是开启的

**叶子节点：**`x`、`w`、`b`、`y`，并非由计算得出的，而是计算的起点，可以通过`is_leaf()`方法判断是否为叶子节点，反向传播计算梯度后，叶子节点中的`grad`属性会一直保存，非叶子节点不会自动保存，除非设置`retain_grad`属性

**根节点：**`loss`

In [2]:
# 定义数据
x = torch.tensor(10.0)
y = torch.tensor([[3.0]])
print(x)
print(y)

tensor(10.)
tensor([[3.]])


In [3]:
# 定义参数，权重和偏置
w = torch.rand(1, 1, requires_grad=True)  # 1 x 1的矩阵，需要计算梯度，开启requires_grad属性
b = torch.rand(1, 1, requires_grad=True)  # 1 x 1的矩阵，需要计算梯度，开启requires_grad属性
print(w)
print(b)

tensor([[0.8231]], requires_grad=True)
tensor([[0.8497]], requires_grad=True)


In [4]:
# 前向传播，得到输出值
z = w * x + b  # z为某些张量得到的一个新的张量
print(z)

tensor([[9.0809]], grad_fn=<AddBackward0>)


In [5]:
print(x.is_leaf)
print(w.is_leaf)
print(b.is_leaf)
print(y.is_leaf)
print(z.is_leaf)

True
True
True
True
False


In [6]:
# 设置损失函数，均方误差
loss = torch.nn.MSELoss()
loss_value = loss(z, y)  # 通过真实值和预测值计算误差
print(loss_value)
print(loss_value.is_leaf)

tensor(36.9775, grad_fn=<MseLossBackward0>)
False


In [7]:
# 反向传播
loss_value.backward()

In [8]:
# 查看梯度
print(w.grad)
print(b.grad)

tensor([[121.6183]])
tensor([[12.1618]])


自动微分的关键就是记录节点的数据与运算。数据记录在张量的`data`属性中，计算记录在张量的`grad_fn`属性中。


计算图根据搭建方式可分为静态图和动态图，`PyTorch`是动态图机制，在计算的过程中逐步搭建计算图，同时对每个`Tensor`都存储`grad_fn`供自动微分使用。


若设置张量参数`requires_grad=True`，则`PyTorch`会追踪素有基于该张量的操作，并在反向传播时计算其梯度。以来于叶子节点的节点，`requires_grad`默认为`True`。当计算到根节点后，在根节点调用`backward()`方法即可反向传播计算计算图中所有节点的梯度。


非叶子节点的梯度在反向传播之后会被释放掉（除非设置参数`retain_grad=True`）。而叶子节点的梯度在反向传播之后会保留（累积）。通常需要`optimizer.zero_grad()`清零参数的梯度。


有时我们希望将某些计算移动到计算图之外，可以使用`Tensor.detach()`返回一个新的变量，该标量与原变量具有相同的值，但丢失计算图中如何计算原变量的信息。换句话说，梯度不会在该变量处继续向下传播。

# 分离张量

有时候我们希望将某些计算移动到计算图之外，可以使用`Tensor.detach()`返回一个新的变量，该变量与原变量有相同的值，但丢失图中如何计算原变量的信息。换句话说，梯度不会在该变量出继续向下传播

In [9]:
x = torch.tensor(2.0, requires_grad=True)
y = x.detach()
print(x.requires_grad)
print(y.requires_grad)
# 不同的对象
print(id(x))
print(id(y))
# 数据存放的位置是相同的
print(x.untyped_storage().data_ptr())
print(y.untyped_storage().data_ptr())

True
False
1945804231680
1947265010896
5262610075648
5262610075648


In [10]:
print(x)
y.add_(10)
print(x)

tensor(2., requires_grad=True)
tensor(12., requires_grad=True)


In [11]:
# 分别对x和y进行后续计算
z1 = x ** 2
z2 = y ** 2
print(z1)
print(z2)

tensor(144., grad_fn=<PowBackward0>)
tensor(144.)


In [12]:
# 将sum()作为损失函数进行反向传播
z1.sum().backward()

In [13]:
# 将sum()作为损失函数进行反向传播
# z2.sum().backward() # 无法进行反向传播  element 0 of tensors does not require grad and does not have a grad_fn

In [14]:
x = torch.ones(2, 2, requires_grad=True)
y = x * x
print(x)
print(y)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[1., 1.],
        [1., 1.]], grad_fn=<MulBackward0>)


In [15]:
# 分离y
u = y.detach()
print(u)

tensor([[1., 1.],
        [1., 1.]])


In [16]:
# # 定义z，让u参与新张量的计算
# z1 = u * x
# # 反向传播，计算梯度
# z1.sum().backward()
# # 查看x的梯度
# print(x.grad)
# print(x.grad == u)

In [17]:
# 让y参与新张量的计算
z2 = y * x
z2.sum().backward()
print(x.grad)

tensor([[3., 3.],
        [3., 3.]])


<hr/>

`.data`和`.detach()`的对比

In [18]:
# 两组数据进行对比
x1 = torch.tensor([1.0, 2, 3], requires_grad=True)
x2 = torch.tensor([1.0, 2, 3], requires_grad=True)
print(x1)
print(x2)

tensor([1., 2., 3.], requires_grad=True)
tensor([1., 2., 3.], requires_grad=True)


In [19]:
y1 = x1.sigmoid()
y2 = x2.sigmoid()
print(y1)
print(y2)

tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>)
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward0>)


In [20]:
# y1.sum().backward()
# print(x1.grad)

In [21]:
# y2.sum().backward()
# print(x2.grad)

In [22]:
z1 = y1.data
z2 = y2.detach()
print(z1)
print(z2)
print(z1.requires_grad)
print(z2.requires_grad)

tensor([0.7311, 0.8808, 0.9526])
tensor([0.7311, 0.8808, 0.9526])
False
False


In [23]:
z1.zero_()
z2.zero_()
print(y1)
print(y2)

tensor([0., 0., 0.], grad_fn=<SigmoidBackward0>)
tensor([0., 0., 0.], grad_fn=<SigmoidBackward0>)


In [24]:
y1.sum().backward()
print(x1.grad)  # 计算流程图没有改变，仅仅通过data修改了数值，导致梯度计算出现错误，不安全

tensor([0., 0., 0.])


In [25]:
y2.sum().backward()
print(x2.grad)  # 报错，自动微分引擎发现数值修改，不允许梯度计算，安全

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3]], which is output 0 of SigmoidBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).